In [ ]:
from pathlib import Path
from pprint import pprint
from ost import Sentinel1Batch
from ost.helpers import vector, raster

# define the project directory
project_dir = Path.home() / "Documents" / "Python" / "sen1_ost" / "data"

# aoi file name
aoi_file = 'test_shape2.geojson'

# create buffer around aoi
vector.buffer_shape(str(project_dir) + '/aoi/' + aoi_file, str(project_dir) + '/aoi/aoi_tmp' ,buffer=0.01)
aoi = vector.aoi_to_wkt(str(project_dir) + '/aoi/aoi_tmp')

# define the start and end date
start = "2020-05-01"
end = "2020-05-31"

# initialize the class to s1_grd instance
s1_grd = Sentinel1Batch(
    project_dir=project_dir,
    aoi=aoi,
    start=start,
    end=end,
    product_type="GRD",
    beam_mode="IW",
    polarisation="*",
    ard_type="OST-RTC", #Radiometrically Terrain Corrected, images are corrected for radiometric distortions along mountainous slopes
    max_workers=8,
)

# search for files
# added scihub username and pw in Project.py line 260
if not s1_grd.inventory_file:
    s1_grd.search()

# refine inventory
s1_grd.refine_inventory()

# download ascending orbit images
key = "ASCENDING_VVVH"
s1_grd.download(s1_grd.refined_inventory_dict[key], concurrent=10, mirror = '2', uname='langdav', pword='SoLaDaSon_160317')